### Como manipular XML com XPath

In [495]:
from gensim.models import KeyedVectors
import unicodedata

import gzip
import xml.etree.ElementTree as ET

with gzip.open('pordesc2018.xml.gz') as pordesc2018:
    tree = ET.parse(pordesc2018)

In [496]:
from unicodedata import normalize
def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

### Monta Dicionario

In [497]:
wordModel = KeyedVectors.load_word2vec_format('health_w2v_unigram_50.bin', binary=True)
cont = 0

dictMesh = {}
indiceReverso = {}
for d in tree.findall("./DescriptorRecord"):
    terms = []
    
    qualifier = '#'
    
    for aql in d.findall('.AllowableQualifiersList/AllowableQualifier/QualifierReferredTo/QualifierName'):
        teste_qualifier = aql.find('./String').text
        if(teste_qualifier == 'anatomy & histology' or teste_qualifier == 'pharmacology' or teste_qualifier == 'methods' or teste_qualifier == 'diagnosis'):
            qualifier = teste_qualifier
            break
    
    for c in d.findall('.ConceptList/'):
        
        termos_similares = []
        porc_lista = []
        porc_medio = 0
        
        if c.find('./ScopeNote') != None:
            scope = c.find('./ScopeNote').text.replace('\n','').strip()
        for t in c.findall('./TermList/'):            
            terms.append(t.find('./String').text)
            cont_porcentagem = 0
            
            palavra_similar = []
            
            if t.find('./String').text.lower() in wordModel.vocab:
                sem_assento = remover_acentos(t.find('./String').text.lower())
                palavra_similar = wordModel.most_similar_cosmul(sem_assento,topn=10)
                for similar, porcentagem in palavra_similar:
                    if(porcentagem > 0.89 and qualifier != 'pharmacology'):
                        termos_similares.append(similar)
                        porc_lista.append(porcentagem)
                        porc_medio += porcentagem
                    elif(porcentagem > 0.9 and qualifier == 'pharmacology'):
                        termos_similares.append(similar)
                        porc_lista.append(porcentagem)
                        porc_medio += porcentagem
                        
        if(len(porc_lista) > 0):
            porc_medio = porc_medio/(len(porc_lista))
            
        if(len(termos_similares) > 0):
            for i in range(len(termos_similares)):
                flag = -1
                porc_maior=0
                if not (termos_similares[i] in indiceReverso):
                    flag = 0
                    terms.append("<i> "+termos_similares[i]+" </i>" + " <input class='radio' type='radio' name='"+termos_similares[i]+"' value='1'/> Certo <input class='radio' type='radio' name='"+termos_similares[i]+"' value='0'/> Errado")
                
                elif(indiceReverso[termos_similares[i]]['porc_similar'] < porc_lista[i]):
                    flag = 1
                    porc_maior = porc_lista[i]
                    terms.append("<i> "+termos_similares[i]+" </i>" + " <input class='radio' type='radio' name='"+termos_similares[i]+"' value='1'/> Certo <input class='radio' type='radio' name='"+termos_similares[i]+"' value='0'/> Errado")
                
                elif(indiceReverso[termos_similares[i]]['porc_similar'] < porc_medio \
                     and indiceReverso[termos_similares[i]]['porc_medio'] < porc_medio):   
                    flag = 2
                    porc_maior = porc_lista[i]
                    if(termos_similares[i] == 'escitalopran' ):
                        print (flag, porc_medio, porc_lista[i])
                    terms.append("<i> "+termos_similares[i]+" </i>" + " <input class='radio' type='radio' name='"+termos_similares[i]+"' value='1'/> Certo <input class='radio' type='radio' name='"+termos_similares[i]+"' value='0'/> Errado")

                if((flag != -1) and (termos_similares[i] in indiceReverso)):
                    ID = indiceReverso[termos_similares[i]]['ID']
                    if(ID in  dictMesh):
                        for t in range(len(dictMesh[ID]['terms'])-1):
                            if '<i> '+termos_similares[i]+' </i>' in dictMesh[ID]['terms'][t]:
                                #print(termos_similares[i])
                                del dictMesh[ID]['terms'][t] 
                
                if(flag != -1):                       
                    indiceReverso[termos_similares[i]+'_i'] = {
                        'ID': d.find('.DescriptorUI').text,
                        'porc_similar': porc_maior,
                        'porc_medio': porc_medio,
                        'flag': flag

                    }  
    
    terms.append(" <input type='hidden' name='ID' value='"+d.find('.DescriptorUI').text+"'/> ")                         
    dictMesh[d.find('.DescriptorUI').text] = {
        'ID': d.find('.DescriptorUI').text,
        'name': d.find('.DescriptorName/String').text,
        'scope': scope,
        'terms': sorted(set(terms), reverse=True),
        'qualifier': qualifier
    }
    
for dui in list(dictMesh):
    d = dictMesh[str(dui)]
    for t in d['terms']:
        if not('<i>' in t):
            indiceReverso[t.lower()] = {
                'ID': d['ID']
            }  
    #break
len(dictMesh)
print(cont)

2 0.9336397171020507 0.9291593432426453
0


### Salva Dicionario

In [498]:
wordModel.most_similar_cosmul('escitalopran'.lower(),topn=10)

[('bromazepan', 0.940631091594696),
 ('aripiprazol', 0.9329935908317566),
 ('lorazepan', 0.9326410293579102),
 ('lorazepam', 0.9291593432426453),
 ('procimax', 0.9282625913619995),
 ('lexotan', 0.92816162109375),
 ('bromazepam', 0.9269586205482483),
 ('escitalopram', 0.926773726940155),
 ('fluxoetina', 0.9229224920272827),
 ('rispiridona', 0.9221132397651672)]

In [499]:
#dictMesh['D008140']

indiceReverso['dor abdominal']

#dolutegravir

#bupropiona

#dapsona D061466 D000069454 D019438


{'ID': 'D015746'}

In [500]:
import gzip, pickle

with gzip.open('indiceReversoPT.dict.gz','wb') as fp:
    pickle.dump(indiceReverso,fp)
    fp.close()

In [501]:
import gzip, pickle

with gzip.open('dictMesh.dict.gz','wb') as fp:
    pickle.dump(dictMesh,fp)
    fp.close()

### Carrega Dicionario

In [502]:
import gzip, pickle

with gzip.open('dictMesh.dict.gz','rb') as fp:
    dictMesh = pickle.load(fp)
    fp.close()
    
dictMesh['D000001']

{'ID': 'D000001',
 'name': 'Calcimicina[Calcimycin]',
 'qualifier': 'pharmacology',
 'scope': 'An ionophorous, polyether antibiotic from Streptomyces chartreusensis. It binds and transports CALCIUM and other divalent cations across membranes and uncouples oxidative phosphorylation while inhibiting ATPase of rat liver mitochondria. The substance is used mostly as a biochemical tool to study the role of divalent cations in various biological systems.',
 'terms': ['Calcimycin',
  'Calcimicina',
  'Antibiotic A23187',
  'A23187, Antibiotic',
  'A23187',
  'A-23187',
  'A 23187',
  " <input type='hidden' name='ID' value='D000001'/> "]}

In [6]:
palavra = 'Calcimicina'
for dui in dictMesh:
    d = dictMesh[dui]
    for t in d['terms']:
        if t.lower() == palavra.lower():
            print(t)
            print(d['scope'])

Calcimicina
An ionophorous, polyether antibiotic from Streptomyces chartreusensis. It binds and transports CALCIUM and other divalent cations across membranes and uncouples oxidative phosphorylation while inhibiting ATPase of rat liver mitochondria. The substance is used mostly as a biochemical tool to study the role of divalent cations in various biological systems.
